I think that an efficient way to find the tags of #book inside my 
whole obsidian vault is by using grep, but as this is a python 
programme, I don't know if I can insert shell execution in here. 
I should probably be able to do so if there is a built library.

In [ ]:
import subprocess
books = ""
# books.join(subprocess.run(['shell', 'rg -r "#book" home/schronding/Documents/clean_sync']))



There is indeed a library called subprocess that is  able to 
access the terminal. It seems that with subprocess.run() I can
run any command from the shell. I think that the command should
look something like 
'rg -r "#book" home/schronding/Documents/clean_sync'


Here my purpose is to insert the list of the books as a string 
inside the variable 'books'. 
FileNotFoundError: [Errno 2] No such file or directory: 'shell'
It seems like there is no 'shell'. It probably goes by another name. 
It seems that what is missing is the argument 'shell=True' in the
method '.run()'

In [ ]:
subprocess.run(['shell', 'rg -r "#book" home/schronding/Documents/clean_sync'], shell=True)

The command seems to work, but I don't know if I am missing the syntax
of ripgrep, or if I wrote the routed incorrectly. 
pwd returned /home/schronding/Documents/clean_sync

In [ ]:
subprocess.run(['shell', 'rg -r "#book" /home/schronding/Documents/clean_sync'], shell=True)


The error 
'rg -r "#book" /home/schronding/Documents/clean_sync: 1: shell: not found' 
persists. I might be messing up with the ripgrep syntax
It seems thar ripgrep uses a syntax similar to sed. The thing I have 
notice is that even if the command works, it will probably just return the 
line that contains the hashtag '#book', but what I want is the title
of the note. I will assume that the titles are on the first line of the
markdown file. I don't how I can narrow the search of ripgrep, but at 
least I found a command that I think will work

In [ ]:

subprocess.run(['shell', 'rg --files | rg #book'], shell=True)

It stil says that the shell hasn't been found
rg --files | rg #book: 1: shell: not found
It seems I indeed need to wrap the searching term between quotes 
so ripgrep can found it. While it seems that the command worked, it 
didn't print anything, which is weird 
rg --files | rg #book
rg: ripgrep requires at least one pattern to execute a search
rg --files | rg "#book"
Nothing
It seems I was overcomplicating things as simply going to the folder where
I want to use rg is enough for the program to look recursively on all the 
children folders. It also provides me the with the title of the note, so I don't
need to do anything special, the correct line of code is 

In [6]:
subprocess.run(['rg "#book" /home/schronding/Documents/clean_sync'], shell = True)

/home/schronding/Documents/clean_sync/4_Archives/SM Learning how to say goodbye/You should have faith in your relationship.md:#book #personal 
/home/schronding/Documents/clean_sync/4_Archives/SM Don't close your heart/You should have faith in your relationship.md:#book #personal 
/home/schronding/Documents/clean_sync/5_Inbox/More or less.md:#book 
/home/schronding/Documents/clean_sync/4_Archives/SM People always make sense/Taking responsibility about everything is dangerous.md:#book 
/home/schronding/Documents/clean_sync/4_Archives/SM People always make sense/You don't need to know everything to help.md:#book 
/home/schronding/Documents/clean_sync/4_Archives/SM People always make sense/We cannot control loyalty of others.md:#book #personal 
/home/schronding/Documents/clean_sync/4_Archives/SM People always make sense/Usefulness is nothing without coherence.md:#book #personal 
/home/schronding/Documents/clean_sync/4_Archives/SM People always make sense/To be free I shouldn't be attatched

KeyboardInterrupt: 

It worked! Now I can see the list of notes that have the tag #book in them. The problem is that it also prints the direction and also the tag at the end. 
I think I might be able to use sed to have just the title of the notes. My idea is to use regex to return the title of the note. Taking what comes after the last 
'/' sounds like a good idea. Erasing what comes after the dot (.) will allow me to remove the tags, as there are some examples in which other tag is put in the same
line as #book

In [7]:
list_of_books = subprocess.run('rg -l "#book" /home/schronding/Documents/clean_sync', shell=True, capture_output=True, text=True)
list_of_books.stdout

"/home/schronding/Documents/clean_sync/2_Areas/books/endure_cameron_hanes/Endure (Cameron Hanes).md\n/home/schronding/Documents/clean_sync/2_Areas/books/ultralearning/Ultralearning.md\n/home/schronding/Documents/clean_sync/2_Areas/books/educated/Educated.md\n/home/schronding/Documents/clean_sync/2_Areas/books/psychedelic_explorer_journey/the psychedelic explorer's guide.md\n/home/schronding/Documents/clean_sync/2_Areas/books/get_better_at_anything/Get better at anything.md\n/home/schronding/Documents/clean_sync/2_Areas/books/doing_good_better/Doing Good Better.md\n/home/schronding/Documents/clean_sync/2_Areas/books/the_sleep_solution/the sleep solution.md\n/home/schronding/Documents/clean_sync/2_Areas/books/everything_is_fucked/Everything is fucked.md\n/home/schronding/Documents/clean_sync/2_Areas/books/meditations_for_mortals/Meditations for mortals.md\n/home/schronding/Documents/clean_sync/2_Areas/books/dont_die/Don't die.md\n/home/schronding/Documents/clean_sync/2_Areas/books/how_to

The magic of using the -l flag is that it gives me the path/name of the file
that contains #book, not the whole line. In this way I save myself the effort of using sed to remove it afterwards. It is the same that '--files-with-matches'
'capture_output=True' is what I needed so python receives the information, as normally the method of .run() just prints to the terminal the result. 
Finally the argument 'text=True' ensures I get a string, not raw bytes. 
What I need to do next is to separate the string into its constituent parts, which seems that can be done with the method '.splitlines()'

In [ ]:
list_of_books.splitlines

It seems that splitlines is not a method of strings, which is curious because the error says that it is an object

AttributeError: 'CompletedProcess' object has no attribute 'splitlines'

I suppose that while I asked for text in the argument there might be some extra information or methods that transform 'list_of_books' into an object. It makes sense as the method .stdout() doesn't exist in strings... does it? 

In [ ]:
test_string = "This is just to try the .stdout() method"
test_string.stdout

Indeed it hasn't 

AttributeError: 'str' object has no attribute 'stdout'

Yet it seems that splitlines() IS a method for strings


In [ ]:
test_string.splitlines()

It indeed does! Then if I can my object list_of_books to string I might be done

In [ ]:
string_of_books = str(list_of_books)
list_of_books = string_of_books.splitlines()
print(list_of_books)

While it worked I don't know if it is what I want, as I see I lot of backslashs, which makes me think there was a lot of escaping, and while I understand it with characters like (') I don't know if I have that many characteres that need to be escaped. I will continue with sed though

In [ ]:
subprocess.run("sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books")

There are a lot of errors, but I think my syntax for sed is correct, the thing is that I am trying to reference a local variable instead of a file. I will save it with json too see how it works

In [ ]:
import json

with open("list_of_books.json", "w") as file:
    json.dump(list_of_books, file, indent=4)

It worked! Now the thing is that it is inside square brackets, and I don't know if that means a literal python list

In [ ]:
subprocess.run("sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books")

It still has an error 

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books"

I think that I need to escape the dashes, as that might be messing up with sed

In [ ]:
subprocess.run("sed -i 's/\/home\/schronding\/Documents\/clean_sync\///g' list_of_books")

It seems that escaping the dashes didn't worked. I think that maybe I could store the path to a variable in order to not worry about messing up with the file and the brackets for substitution

In [ ]:
clean_path = "/home/schronding/Documents/clean_sync/"

Now let's see if I can use f strings with subprocess.run()

In [ ]:
subprocess.run(f"sed -i 's/{clean_path}//g' list_of_books.json")

It seems that my problem came from trying to use the method splitlines() directly in the object, or in the object transformed to string. The correct solution was to use it on the string INSIDE the object, the one I get with the method .stdout

The problem I got with the code line above (subprocess.run(f"sed -i 's/{clean_path}//g' list_of_books.json")) was caued because when you pass a single string to subprocess.run() Python expects that string to be the name of a single file it can execute (I suppose a bash one with multiple instructions). 

In order for the system to correctly interpret the spaces as command arguments I needed to add 'shell=True' as I did on the command with ripgrep previously. 

It seems that I can use any character as a delimiter in sed, so in order to not get confused with the / that indicate path folders on the string, I could just use | or #... whatever character it is not on my particular string. 

It seems that I can use the library 'os' with its module 'path' to handle more easily the strings that represent the file locations. 

With 'os.path.basename(path)' I can return the final component of a path (the actual name of the file), while with 'os.path.dirname(path)' I get returned the directory component of a path. Concatening the dirname and basename would give the original string all together. 




In [13]:
import os

raw_output = list_of_books.stdout
path_list = raw_output.splitlines()
filenames = []
for file in path_list:
    filenames.append(os.path.basename(file))
print(filenames)

['Endure (Cameron Hanes).md', 'Ultralearning.md', 'Educated.md', "the psychedelic explorer's guide.md", 'Get better at anything.md', 'Doing Good Better.md', 'the sleep solution.md', 'Everything is fucked.md', 'Meditations for mortals.md', "Don't die.md", 'How to take smart notes.md', 'the simple path to wealth.md', 'Essentialism.md', 'You can have wealthy girlfriends.md', 'We should reward people when we like something they do.md', 'You can encourage behavior by attention and reward.md', 'You should trust your partner.md', 'Time out can be an effective behavior correction.md', 'We can attract misfortune by trusting carelessly.md', 'When making peace with someone limit the amount of fights.md', 'Wanting a fair exchange is wise, not selfish.md', 'Strategy to meet new people.md', 'Search for necessary pain.md', 'We intuitively identify what is wrong.md', 'We might be condemned to irrationalities.md', 'We cannot be evil while being grateful.md', 'Woman are not the problem, I am.md', 'We ca

Now it is done! What follows is to remove all the .md of the names, which I am pretty sure can be done with os too. But now that I think about it, as every file ends up with .md I can just create a loop and remove the last 3 characters of the string

In [14]:
for i in range(len(filenames)):
    book = filenames[i]
    filenames[i] = book[:-3]

print(filenames)

['Endure (Cameron Hanes)', 'Ultralearning', 'Educated', "the psychedelic explorer's guide", 'Get better at anything', 'Doing Good Better', 'the sleep solution', 'Everything is fucked', 'Meditations for mortals', "Don't die", 'How to take smart notes', 'the simple path to wealth', 'Essentialism', 'You can have wealthy girlfriends', 'We should reward people when we like something they do', 'You can encourage behavior by attention and reward', 'You should trust your partner', 'Time out can be an effective behavior correction', 'We can attract misfortune by trusting carelessly', 'When making peace with someone limit the amount of fights', 'Wanting a fair exchange is wise, not selfish', 'Strategy to meet new people', 'Search for necessary pain', 'We intuitively identify what is wrong', 'We might be condemned to irrationalities', 'We cannot be evil while being grateful', 'Woman are not the problem, I am', 'We can work on the systems we inherited', 'You are the piece that determine how your r

It worked!. Now what I need is to know how can I look for each of these entries in a page to retrieve all the meaningful data I need for my obsidian. It seems that the community plug in uses Google and that you can search on a specific "locale", which I suppose is just from which part of the world you're starting the Google search. 

At the end it says that it uses an API, particularly the Google Books API. 

It also seems that the last commit of the repo was done 2 years ago... which is strange as the Lastest release says that it is on Oct 16 2024. I suppose it is technically true because we're on 2026. 

As it was created on TypeScript I don't think I can contribute with python. 
Maybe I should've done this project with it, as Viktor told me it was a essential skill for me to learn. 

It seems that the Google API might be a bit difficult for me now... Probably the fastest route would just to use this list, insert the metadata from the plug in each of the notes and then review them myself manually to see if they're correct. 

For now I think I can simply save the variable into a file in order to seek it later. 

In [16]:
with open("clean_list_of_books.txt", "w") as file:
    for name in filenames:
        file.write(str(name) + "\n")